# SVM을 사용한 얼굴 검출

- 런타임 유형: 하드웨어 가속기=None

- 출처: https://jakevdp.github.io/PythonDataScienceHandbook/05.14-image-features.html
- modified by 김보연

In [ ]:
from skimage import data, color, feature
import skimage.data
import matplotlib.pyplot as plt
import numpy as np

# 영상과 filter 개념

- https://scikit-image.org/docs/stable/api/skimage.data.html

In [ ]:
image = data.chelsea()
plt.imshow(image)
plt.title('input image')

- https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_hog.html

In [ ]:
image = color.(data.chelsea())
hog_vec, hog_image = feature.(image, visualize=True)

print(hog_image.max(), hog_image.min(), image.min(), image.max())
print(type(hog_vec), hog_vec.shape, hog_vec[:5])

fig, ax = plt.subplots(1, 2, figsize=(12, 6), 
                      subplot_kw = dict(xticks=[], yticks=[]))
ax[0].imshow(image, cmap='gray', vmin=0, vmax=1)
ax[0].set_title('input image')

ax[1].imshow(hog_image, cmap='gray')
ax[1].set_title('Visualization of HOG features')

# 안면 인식 알고리즘 구현: linear SVM 사용

In [ ]:
#얼굴 데이터 추출  (최대 1분소요)
from sklearn.datasets import fetch_lfw_people
faces = fetch_lfw_people()

In [ ]:
#1. 긍정 훈련 표본  
positive_patches = faces.
positive_patches.shape

In [ ]:
print(type(faces))
print(faces.target_names)
print(faces.images.shape)

In [ ]:
skip = 1000
fig, axes = plt.subplots(3,8, figsize=(9,4),
                         subplot_kw={'xticks':[], 'yticks':[]},
                        gridspec_kw=dict(hspace=0.1, wspace=0.1))
for i, ax in enumerate(axes.flat):
    ax.imshow(positive_patches[i+skip], cmap='bone')
plt.show()                            

In [ ]:
positive_patches[0].shape

In [ ]:
#2. 부정 훈련 표본
from skimage import data, transform
images_to_use = ['camera', 'text', 'coins', 'moon', 'page', 'clock', 
                 'immunohistochemistry','chelsea', 'coffee', 'hubble_deep_field']
images = [getattr(data, images_to_use[i])()  for i in range(6)]
images = [color.rgb2gray(getattr(data, images_to_use[i])() )
          for i in range(6, 10)]
print(len(images))

fig, axes = plt.subplots(2,5, figsize=(9,4),
                         subplot_kw={'xticks':[], 'yticks':[]},
                        gridspec_kw=dict(hspace=0.1, wspace=0.1))                    
for i, ax in enumerate(axes.flat):
    ax.imshow(images[i], cmap='bone')
plt.show()

In [ ]:
from sklearn.feature_extraction.image import PatchExtractor

def extract_patches(img, N, scale=1.0, patch_size=positive_patches[0].shape):
    extracted_patch_size = \
     ((scale * np.array(patch_size)).astype(int))
    extractor = PatchExtractor(patch_size=extracted_patch_size,
                              max_patches=N, random_state=0)
    patches = extractor.transform(img[np.newaxis])
    if scale != 1:
        patches = np.array([transform.(patch, patch_size)
                           for patch in patches])
    #print(patches.shape)
    return patches

negative_patches = ([extract_patches(im, 1000, scale)
                            for im in images for scale in [0.5, 1.0, 2.0]])
negative_patches.shape

In [ ]:
fig, ax = plt.subplots(6, 10, figsize=(20, 12))
for i, axi in enumerate(ax.flat):
    axi.imshow(negative_patches[500*i], cmap='gray')
    axi.axis('off')

In [ ]:
# 집합을 결합하고 HOG 특징 추출
from itertools import chain

X_train = np.array([feature.hog(im)
                   for im in chain(positive_patches, negative_patches)])
y_train = (X_train.shape[0])
y_train[:positive_patches.shape[0]] = 

In [ ]:
X_train.shape

In [ ]:
# Naive Bayes 의 얼굴 분류 성능
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

cross_val_score(GaussianNB(), X_train, y_train)

In [ ]:
np.sum(cross_val_score(GaussianNB(), X_train, y_train))/5

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

grid =(LinearSVC(), {'C':[1.0, 2.0, 4.0, 8.0]})
grid.fit(X_train, y_train)
grid.best_score_

- https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html

In [ ]:
grid.best_params_

In [ ]:
model = grid.best_estimator_
model.fit(X_train, y_train)

In [ ]:
test_image = skimage.data.astronaut()
plt.imshow(test_image)

In [ ]:
test_image = skimage.color.(test_image)
plt.imshow(test_image, cmap='gray')

In [ ]:
test_image = skimage.transform.(test_image, 0.5)
test_image = test_image[:160, 40:180]

plt.imshow(test_image, cmap='gray')

In [ ]:
def sliding_window(img, patch_size=positive_patches[0].shape,
                  istep=2, jstep=2):
    
    Ni, Nj = patch_size

    for i in range(0, img.shape[ ] - Ni, istep): #Ni:rows
        for j in range(0, img.shape[ ] - Nj, jstep):  #Nj:columns
            patch = img[i:i+ , j:j+ ]
             (i, j), patch  # generator

In [ ]:
indices, patches = zip( sliding_window(test_image))
patches_hog = np.array([feature.hog(patch) for patch in patches])
patches_hog.shape

In [ ]:
labels = model.predict(patches_hog)
labels.sum()

In [ ]:
fig, ax = plt.subplots()
ax.imshow(test_image, cmap='gray')
ax.axis('off')

Ni, Nj = positive_patches[0].shape
indices = np.array(indices)

idxList=[]
for i, j in indices[labels  ]:
    ax.add_patch(plt.Rectangle((j,i), Nj, Ni, edgecolor='red',
                              alpha=0.3, lw=2, facecolor='none'))
    idxList.append([ , ])   # 사각형의 중앙 위치를 찾기 위함

In [ ]:
print(idxList)

In [ ]:
idx = (idxList)

In [ ]:
idx.mean(axis= )

In [ ]:
i, j = 
i, j

In [ ]:
Ni, Nj = positive_patches[0].shape
Ni, Nj

In [ ]:
fig, ax = plt.subplots()
ax.imshow(test_image, cmap='gray')
ax.axis('off')
Ni, Nj = positive_patches[0].shape
ax.add_patch(plt.Rectangle( (j,i), Nj, Ni, edgecolor='red',
                              alpha=0.5, lw=5, facecolor='none'))
plt.show()

In [ ]:
plt.imshow(test_image[  ,  ], cmap='gray')
plt.show()